# RAG for narration

The basic RAG framework providing by LangChain

## Part 1: Enviornment

`(1) LangSmith`

In [32]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = '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'

`(2) API Keys`

In [33]:
API_SECRET_KEY = "sk-fQmZ27DLJestJOeZ915dEbA0325d47AaAc86B6Ed609625E5"
BASE_URL = "https://api.gpts.vin/v1"

os.environ["OPENAI_API_KEY"]= API_SECRET_KEY
os.environ["OPENAI_API_BASE"]= BASE_URL

## Part 2: Indexing

`Document Preparation`

In [34]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

In [35]:
# blog_docs

* Can add `Routing` method to locate which `Data Source`(Sports, News, Life...) to be used

`Token counter`

In [36]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """
    Returns the number of tokens in a text string.
    """
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [37]:
# num_tokens_from_string(question, "cl100k_base")

`Spliting Document`

In [38]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)

In [39]:
# splits

`Save In VectorStores` (In this process contain `Embedding`)

In [40]:
# import
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# load it into Chroma
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_function)

c:\Users\87898\anaconda3\envs\rag\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Part 3: Retrieval

`Similarity Matching`

In [41]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

`RAG-Fusion`(Query Translation)

In [42]:
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# RAG-Fusion: Related
template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)

generate_queries = (
    prompt_rag_fusion 
    | ChatOpenAI(temperature=0)
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [43]:
from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=60):
    """ 
        Reciprocal_rank_fusion that takes multiple lists of ranked documents 
            and an optional parameter k used in the RRF formula 
    """
    
    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results

retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion
docs = retrieval_chain_rag_fusion.invoke({"question": "What is Task Decomposition?"})
len(docs)

6

In [44]:
docs

[(Document(page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via

## Part 4: Generation

Final `Prompt Template Design`

In [45]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

Specified the `LLM Model`

In [46]:
# LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Chain
chain = prompt | llm

`Final Answer`

In [47]:
# Run
chain.invoke({"context":docs,"question":"What is Task Decomposition?"})

AIMessage(content='Task decomposition is the process of breaking down a complicated task into smaller and simpler steps, which can be achieved through techniques such as Chain of Thought (CoT) and Tree of Thoughts. It can be done by using simple prompting, task-specific instructions, or human inputs.', response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 1601, 'total_tokens': 1655}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c9f32338-1e7d-4b8a-9a5d-e6e4ce9b8b1c-0')

`RAG Chain`

In [ ]:
# from langchain_core.output_parsers import StrOutputParser
# from langchain_core.runnables import RunnablePassthrough

# rag_chain = (
#     {"context": retriever, "question": RunnablePassthrough()}
#     | prompt
#     | llm
#     | StrOutputParser()
# )

# rag_chain.invoke("What is Task Decomposition?")

In [ ]:
'''
import os
import requests
import time
import json
import time
from langchain_openai import OpenAI

API_SECRET_KEY = "sk-fQmZ27DLJestJOeZ915dEbA0325d47AaAc86B6Ed609625E5"
BASE_URL = "https://api.gpts.vin/v1"

os.environ["OPENAI_API_KEY"]= API_SECRET_KEY
os.environ["OPENAI_API_BASE"]= BASE_URL

llm = OpenAI(temperature=0.9)
text = "hello"
llm.invoke(text)
'''